In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

## Prearacion de los datos

Aquí construiremos vectores de 1 y 0 marcando a cada persona si ha ido de viaje a un pais o no

In [2]:
data = pd.read_table('/home/pc/Escritorio/TFM/marketing_pax_clean.csv',sep='^',header=0,decimal='.',index_col=0)

/home/pc/.local/lib/python2.7/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
data.head()

,rloc,full_name,departure_date_leg,board_point,board_country_code,off_point,off_country_code
1,612666,Z/MZFFUFGDGZQ,2014-04-01,BRU,BE,LNZ,AT
3,827666,T/HUYMXMTGOTMUM,2013-02-02,BRU,BE,LIL,FR
4,8PD666,YU/YUDVMMTHQZEMX,2013-10-25,BRU,BE,MMX,SE
7,0QB666,VAZMFTMZBQSXQD/,2014-04-01,BRU,BE,KTM,NP
10,13J666,QZ/WMDUYUQFFUZ,2013-01-17,ALG,DZ,BRU,BE


In [4]:
#agrupamos todos los viajes que realiza una persona en un mismo registro

In [5]:
grouped = data.groupby('full_name')['off_country_code'].apply(lambda x: "%s" % '|'.join(x)).reset_index()

In [6]:
grouped.head()

,full_name,off_country_code
0,ADHAWWU YMDSMDQFMOTDUEFQZEQZ/FGGXM,BE|SE
1,AEEU MZFQDAXMTFU/VAGZU,DZ
2,AEOMD IUXTQXYWMDXEEAZ/MMWQ,SE|BE
3,AEOMD SGZZMDEEDQUZTMDP/NVADZ,DE|BE
4,AEOMD VATMZZQEYMDFUZEEAZ/XMDE,BE


In [7]:
#convertimos cada regristro de todos los viajes en un vector de 0,1 marcando si ha ido de viaje al pais o no
#para ello utilizamos la funcion get_dummies

In [8]:
datos = pd.concat([grouped,grouped.off_country_code.str.get_dummies(sep = '|')],axis = 1)

In [9]:
datos.head()

,full_name,off_country_code,AT,BE,BG,CA,CH,CN,CV,CY,...,PT,RU,SA,SE,TN,TR,UA,US,UZ,ZM
0,ADHAWWU YMDSMDQFMOTDUEFQZEQZ/FGGXM,BE|SE,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,AEEU MZFQDAXMTFU/VAGZU,DZ,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AEOMD IUXTQXYWMDXEEAZ/MMWQ,SE|BE,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,AEOMD SGZZMDEEDQUZTMDP/NVADZ,DE|BE,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AEOMD VATMZZQEYMDFUZEEAZ/XMDE,BE,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Cargo los datos del modelo en la base de datos

In [10]:
import sqlite3 as sql3
path_db = '/home/pc/Escritorio/TFM/MyDestiny/MyDestinyAPP/MyDestiny/app.db'
index = 'id'

conn = sql3.connect(path_db)

#datos.to_sql(name='datosmodelo', index_label=index, con=conn,if_exists='replace')

## Creacion de raiting y recomendacion de viaje

Ahora lo que vamos hacer es coger y agrupar todos los vectores que sean de la misma forma y contar cuantas veces aparecen, ese conteo sera el peso o raiting del vector.
Una vez agrupados haremos un kneighbours para ver que vectores son los mas parecidos al que queremos predecir.
Finalmente con los vectores parecidos calcularemos el coeficiente de cada pais de posible destino como la peso del vector dividio por la distancia al vector ogirinal y el pais que mas coeficiente saque será el recomendado

un ejemplo sencillo de lo que se hace acontinuacion se puede ver el el notebook modelo_parte_2 que hace un caso sencillo de como se realiza lo descrito anteriormente

In [11]:
#agrupamos todos los vectores iguales

In [12]:
destinos = datos.columns[2:].tolist()
agrupados = datos.groupby(destinos)['full_name'].count().reset_index()
agrupados.head()

,AT,BE,BG,CA,CH,CN,CV,CY,DE,DZ,...,RU,SA,SE,TN,TR,UA,US,UZ,ZM,full_name
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,292
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,212
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,3875
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,4720


## Crear la base de datos de agrupados

In [13]:
#agrupados.to_sql(name='agrupados_modelo', index_label=index, con=conn,if_exists='replace')

In [14]:
#preparamos los datos para entrenar el modelo
X= agrupados[destinos].values
X

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ..., 
       [1, 1, 1, ..., 1, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 0]])

In [15]:
#entrenamos el modelo
vecinos  = NearestNeighbors(n_neighbors=5,metric='cosine',algorithm='brute')
vecinos = vecinos.fit(X)


In [16]:
#este sera el valor a predecir
#datos.iloc[8,:]
predecir = np.array([datos[destinos].values[25]])
datos.iloc[25,:]

full_name            AGFU TMZZQXQYGEFAZQZ/YUUM
off_country_code                         BE|JE
AT                                           0
BE                                           1
BG                                           0
CA                                           0
CH                                           0
CN                                           0
CV                                           0
CY                                           0
DE                                           0
DZ                                           0
ES                                           0
FR                                           0
GB                                           0
GE                                           0
GL                                           0
GR                                           0
IE                                           0
IL                                           0
IQ                                           0
IS           

In [17]:
#sacamos las prediciones, o vectores que mas se asemejen al original
distances, indices = vecinos.kneighbors(predecir)
#serian los vecinos
agrupados.iloc[indices[0],:]


,AT,BE,BG,CA,CH,CN,CV,CY,DE,DZ,...,RU,SA,SE,TN,TR,UA,US,UZ,ZM,full_name
1267,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,30771
16274,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,57
1314,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,100
1701,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,277
1454,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,26


In [18]:
#funcion auxiliar para restar dos listas coordenada a coordenada
def restar(a,b):
    lon = len(a)
    sol = [0]*lon
    for i in range(lon):
        sol[i] = a[i]-b[i]
    return sol

In [19]:
#funcion auxiliar para sumar dos listas coordenada a coordenada
def suma(a,b):
    lon = len(a)
    sol = [0]*lon
    for i in range(lon):
        sol[i] = a[i]+b[i]
    return sol

In [20]:
#funcion que calcula el pais con mas rating y acaba recomendandonoslo
def dar_destino(listaCandidatos,pesos,distances,predecir):
    destinos = []
    for i in listaCandidatos:
        aux = restar(i,predecir)
        destinos.append(aux) 
    destinoPesos = []
    for i in range(len(pesos)):
        distancia = distances[i]
        peso = pesos[i]
        destino = destinos[i]
        destinoPesos.append(map(lambda x: x*peso,destino))
    sol = reduce(lambda x,y:suma(x,y),destinoPesos,[0]*len(destinoPesos[0]))
    destino = sol.index(max(sol))
    return destino

In [21]:
#definimos las variables para sacar resultado
candidatos = agrupados.iloc[indices[0],:][destinos].values
pesos = agrupados.iloc[indices[0],:]['full_name'].values

In [22]:
#obtenemos el resultado
destino = dar_destino(candidatos,pesos,distances[0],predecir[0])
nombreisodestino = destinos[destino]

In [88]:
#funcion que calcula el pais con mas rating y acaba recomendandonoslo
def dar_destino_top5(listaCandidatos,pesos,distances,predecir):
    destinos = []
    for i in listaCandidatos:
        aux = restar(i,predecir)
        destinos.append(aux) 
    destinoPesos = []
    for i in range(len(pesos)):
        distancia = distances[i]
        peso = pesos[i]
        destino = destinos[i]
        destinoPesos.append(map(lambda x: x*peso,destino))
    sol = reduce(lambda x,y:suma(x,y),destinoPesos,[0]*len(destinoPesos[0]))
    orden = sorted(sol)
    orden.reverse()
    top_5 = orden[0:5]
    destino = [sol.index(i) for i in top_5]
    return destino

In [89]:
destino5 = dar_destino_top5(candidatos,pesos,distances[0],predecir[0])

In [90]:
destino5

[20, 34, 4, 26, 0]

In [91]:
nombreisodestino5 = [destinos[i] for i in destino5]
nombreisodestino5

['IT', 'PT', 'CH', 'LK', 'AT']

In [92]:
nombreisodestino

'IT'

Leemos de wikipedia una tabla con los nombres de los paises y sus correspondientes codigos iso, para poder traducir el resultado que viene en codigo iso, al nombre mas comun del pais

In [23]:
#codigos alfa-2
tables = pd.read_html('https://es.wikipedia.org/wiki/ISO_3166-1')
tabla = tables[1]
columnas = tabla.iloc[0,:]
tabla.columns = ['nombre_comun','nombre_oficial','alfa-2','alfa-3','cod_numerico','obervaciones']
codigosPaises = tabla[1:]

In [24]:
codigosPaises.head()

,nombre_comun,nombre_oficial,alfa-2,alfa-3,cod_numerico,obervaciones
1,Afganistán,Afganistán,AF,AFG,004,NaN
2,Åland,"Åland, Islas",AX,ALA,248,Es una provincia autónoma de Finlandia.
3,Albania,Albania,AL,ALB,008,NaN
4,Alemania,Alemania,DE,DEU,276,Códigos obtenidos del idioma nativo (alemán): ...
5,Andorra,Andorra,AD,AND,020,NaN


# Devolvemos la informacion del pais

In [25]:
destinoFinal = codigosPaises[codigosPaises['alfa-2']==nombreisodestino]

In [26]:
destinoFinal

,nombre_comun,nombre_oficial,alfa-2,alfa-3,cod_numerico,obervaciones
118,Italia,Italia,IT,ITA,380,NaN
